In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
torch.__version__


Data reading and preparing

In [ ]:
weight = 0.7
bias =0.3
start =0
end = 1
step = 0.02
X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight * X + bias

X[:10],y[:10]

In [ ]:
trainsplit = int(0.8*len(X))
xtrain,ytrain = X[:trainsplit],y[:trainsplit]
xtest,ytest = X[trainsplit:],y[trainsplit:]
len(xtrain),len(xtest),len(ytrain),len(ytest)

In [ ]:
def plotpredictions(traindata=xtrain,trainlabels=ytrain,testdata=xtest,testlabels=ytest,predictions=None):
    plt.figure(figsize=(10,7))
    plt.scatter(traindata,trainlabels,c="b",s=4,label="training data")
    plt.scatter(testdata,testlabels,c="g",s=4,label="test data")
    if predictions is not None:
        plt.scatter(testdata,predictions,c="r",s=4,label="predictions")
    plt.legend(prop={"size":15})

#pytorch model building essentials
*torch.nn -> contains all buildings for computing graphs(NN)
*torch.nn.parameter -> what params should our model try
*torch.nn.Module -> base class for all NN modules
* def forward() -> all nn.Module subclassess should overwrite forward() ,this will define what will happen in further computation

In [ ]:
class LinearRegressionModel(nn.Module):#everything in pytorch inherits from nn.Module
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,requires_grad=True,dtype=torch.float32))
        self.bias = nn.Parameter(torch.randn(1,requires_grad=True,dtype=torch.float32))
        #nn.Parameter is a special type of tensor that is a module parameter.
    def forward(self,x:torch.Tensor)->torch.Tensor:
        return self.weights * x + self.bias



In [ ]:
torch.manual_seed(42)

In [ ]:
model0 = LinearRegressionModel()
list(model0.parameters())

In [ ]:
model0.state_dict()

In [ ]:
#making prediction using torch.inference_mode()

with torch.inference_mode():
    ypred = model0(xtest)
ypred

In [ ]:
plotpredictions(predictions=ypred)

In [ ]:
#loss function
lossfn = nn.L1Loss()


In [ ]:
##optimizer
optimizer = torch.optim.SGD(model0.parameters(),lr=0.01)

In [ ]:
torch.manual_seed(42)
epochs =600
epochcount =[]
lossvalues=[]
test_loss =[]

for epoch in range(epochs):
    model0.train()
    ypred = model0(xtrain)
    loss = lossfn(ypred,ytrain)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model0.eval()
    with torch.inference_mode():
        ypred = model0(xtest)
        testloss = lossfn(ypred,ytest)
    epochcount.append(epoch)
    lossvalues.append(loss)
    test_loss.append(testloss)
    print(f"epoch {epoch+1} test loss {testloss}")

print(model0.state_dict())

In [ ]:
plotpredictions(predictions=ypred)

In [ ]:
plt.plot(epochcount,np.array(torch.tensor(lossvalues).numpy()),label="training loss")
plt.plot(epochcount,test_loss,label="test loss")
plt.legend()
plt.show()

In [ ]:
with torch.inference_mode():
    yprednew = model0(xtest)

In [ ]:
plotpredictions(predictions=yprednew)

<center> saving model in pytorch

In [ ]:
from pathlib import Path

modelpath = Path("model")
modelpath.mkdir(parents=True,exist_ok=True)

modelname = "basicPTmodel.pth"
modelsavepath = modelpath/modelname

torch.save(obj=model0.state_dict(),f=modelsavepath)

modelsavepath


<center> loading a model

In [ ]:
loadedmodel = LinearRegressionModel()
loadedmodel.load_state_dict(torch.load(f=modelsavepath))

In [ ]:
loadedmodel.state_dict()

In [ ]:
loadedmodel.eval()
with torch.inference_mode():
    ypredloaded = loadedmodel(xtest)

In [ ]:
yprednew == ypredloaded

In [ ]:
ypredloaded,yprednew

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"using {device} device")

In [ ]:
#data
weight = 0.7
bias = 0.3
start =0
end = 1
step = 0.02
X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight * X + bias

X[:10],y[:10]

In [ ]:
#split data
trainsplit = int(0.8*len(X))
xtrain,ytrain = X[:trainsplit],y[:trainsplit]
xtest,ytest = X[trainsplit:],y[trainsplit:]
len(xtrain),len(xtest),len(ytrain),len(ytest)

In [ ]:
#plot the data to visualize the data
def plotpredictions(traindata=xtrain,trainlabels=ytrain,testdata=xtest,testlabels=ytest,predictions=None):
    plt.figure(figsize=(10,7))
    plt.scatter(traindata,trainlabels,c="b",s=4,label="training data")
    plt.scatter(testdata,testlabels,c="g",s=4,label="test data",marker="x")
    if predictions is not None:
        plt.scatter(testdata,predictions,c="r",s=4,label="predictions")
    plt.legend(prop={"size":15})

In [ ]:
plotpredictions()

In [ ]:
#Pytorch linear model
class LinearRegressionModelv2(nn.Module):
    def __init__(self):
        super().__init__()
        self.linearlayer = nn.Linear(in_features=1,out_features=1,bias=True)
    def forward(self,x:torch.Tensor)->torch.Tensor:
        return self.linearlayer(x)

torch.manual_seed(42)
model1 = LinearRegressionModelv2().to(device)
model1.state_dict()

In [ ]:
next(model1.parameters()).device

In [ ]:
#loss function
lossfunction = nn.L1Loss()

optimizer = torch.optim.SGD(params=model1.parameters(),lr=0.01)

In [ ]:
torch.manual_seed(42)
epochs = 200

xtrain,ytrain = xtrain.to(device),ytrain.to(device)
xtest,ytest = xtest.to(device),ytest.to(device)

for epoch in range(epochs):
    model1.train()
    ypred = model1(xtrain.to(device))
    loss = lossfunction(ypred,ytrain.to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model1.eval()
    with torch.inference_mode():
        ypred = model1(xtest.to(device))
        testloss = lossfunction(ypred,ytest.to(device))
    print(f"epoch {epoch+1} test loss {testloss}")


In [ ]:
model1.state_dict(),ypred

In [ ]:
#evaluatio=ng model
model1.eval()

with torch.inference_mode():
    ypred = model1(xtest)
ypred

In [ ]:
plotpredictions(predictions=ypred.cpu())

In [ ]:
#saving and reloading
from pathlib import Path

modelpath = Path("model")
modelpath.mkdir(parents=True,exist_ok=True)

modelname = "model1.pth"
modelsavepath = modelpath/modelname

torch.save(obj=model1.state_dict(),f=modelsavepath)

modelsavepath


In [ ]:
loadmdel1 = LinearRegressionModelv2()
loadmdel1.load_state_dict(torch.load(f=modelsavepath))
loadmdel1.to(device)

In [ ]:
loadmdel1.state_dict()

In [ ]:
loadmdel1.eval()
with torch.inference_mode():
    loadedypred = loadmdel1(xtest.to(device))
ypred==loadedypred

In [ ]:
plotpredictions(predictions=loadedypred.cpu())